# CV Parser

## 1. Introducción

Este segundo notebook tiene como objetivo realizar una segunda lectura de los PDFs. A diferencia de la primera lectura, esta se va a realizar por bloques (chunks) de texto, es decir, utilizandola librería OpenCV y Itext de nuevo, podremos ser capaces de identificar aquellas zonas del pdf que contenga texto, agruparlas en bloques y obtener propiedades como la posición y su contenido. Los resultados, se volverán a guardar en MongoDB

## 2. Pasos

1. Lectura repositorio MongoDB
2. Obtención de chunks y características - OpenCV/Itext
3. Exportación de resultados a MongoDB

## 3. Código fuente

### 3.1 Lectura repositorio MongoDB

En primer lugar, hacemos uso del repositorio creado en el primer Notebook para cargar nuestro dataset en memoria

In [2]:
from pymongo import MongoClient

In [3]:
myclient = MongoClient("mongodb://localhost:27017/")
mydb = myclient["candidates"]
mycol = mydb["CV"]

A partir de este punto, el dataset siempre va a ser filtrado por aquellos CVs que se hayan identificado como tipo texto y sobretodo, en idioma español, con el objetivo de facilitar la extracción de información en los pasos posteriores.

In [5]:
myquery = {"Type":"Text","Language":"Spanish"}
mydoc = mycol.find(myquery)

### 3.2 Obtención de chunks y características - OpenCV/Itext

Para identificar los chunks de texto, se va a usar la librería OpenCV y Sklearn que a través del algoritmo DBSCAN, extraerá las coordenadas de aquellas zonas del pdf consideradas como bloques de texto.

In [6]:
from Utilities import *
import tempfile
import numpy as np

Cada chunk de texto, contendrá la siguiente información:

- id: identificador numérico del chunk
- text: texto contenido en el chunk
- x: coordenada x con respecto al pdf
- y: coordenada y con respecto al pdf
- width: ancho del chunk
- height: largo del chunk
- type: campo que se utilizará más adelante para clasificar al chunk
- parent: campo que se utilizará más adelante para relaccionar chunks
- n_words: el total de palabras encontradas en el chunk
- n_characters: el total de letras encontradas en el chunk

In [7]:
dic_keys = ["id","text","x","y","width","height","type","parent","n_words","n_characters"]
chunksDic = dict((el,0) for el in dic_keys)

Con las dimensiones del chunk extraídas, nos encontramos con dos alternativas para extraer el contenido:

1. Utilizar un OCR: la lectura se realizaría a través del análisis de la imagen
2. Utilizar Itext: esta librería permite leer un pdf por zonas a través de coordenadas, evitando tener que hacer uso de imágenes

La opción elegida es la número 2 pues es más fiable al trabajar directamente con el pdf digital y no con imágenes. Además, la velocidad de procesamiento es muyo mayor

In [8]:
import clr
clr.AddReference(r'C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\Itext\Itext-CVs.dll')
from wavespace import Pdf
pdfParser = Pdf()
pdfParser.setEncoding()

In [9]:
index = 0

from nltk.tokenize import RegexpTokenizer
import re
tokenizer = RegexpTokenizer(r'\w+')

for CV in mydoc:
    
    index = index + 1
    chunks = []
    err = False
    
    if not 'Chunks' in CV:
        with open(CV['File'], "rb") as f:

            try:
                tempf = tempfile.NamedTemporaryFile(delete=False)
                tempf.write(f.read())
                imgs_lists = generate_images(tempf,CV['Pages'])
            except:
                err = True

            if not err:         
                #IMAGES ARE SCALATED TO 3840, 2160
                imgs,escalador = scale_img(imgs_lists)

                #IMAGES ARE PREPROCESSED TO IMPROVE THE CHUNKS ACCURACY
                imgs2 = preprocess_images(imgs)

                #Open PDF in ITEXT
                pdfParser.openPDF(CV['File'])
                pdf_h = pdfParser.getSizeTop(1)
                pdf_w = pdfParser.getSizeRight(1)

                try:
                    img_chunk_list = cluster_extraction(imgs2,thres_x=20, page_type ="hr")
                except:
                    pass

                #IMAGES ARE SCALATED TO A4 (842,595) to adapt them to Itext    
                imgs,escalador = scale_img(imgs,(pdf_h,pdf_w))

                id_ = 0
                for idx, page in enumerate(img_chunk_list):
                    for chunk in page:

                        (x, y) = chunk['position']
                        (w, h) = chunk['size']  

                        content = ""
                        try:
                            content = pdfParser.readPDFbyArea(idx + 1,x*escalador,y*escalador,w*escalador,h*escalador)                    
                            if content.strip() == "":
                                for pixeles in range(1,3):
                                    content = pdfParser.readPDFbyArea(idx + 1,x*escalador,(y*escalador)-pixeles,w*escalador,(h*escalador)+(pixeles*2))
                                    if content.strip() != "":
                                        break
                                    else:
                                        pixeles += 1 
                        except:
                            pass

                        if content != "":           
                            chunksDic["id"] = id_
                            chunksDic["x"] = x
                            chunksDic["y"] = y
                            chunksDic["width"] = w
                            chunksDic["height"] = h 
                            chunksDic["type"] = "" 
                            chunksDic["parent"] = ""
                            chunksDic["n_words"] = len(tokenizer.tokenize(content))
                            chunksDic["text"] = content
                            chunksDic["n_characters"] = len(content.strip())  
                            id_ += 1

                            chunks.append(chunksDic.copy())
                mycol.update_one({"_id": CV["_id"]}, {"$set": {"Chunks":chunks}})

C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\a\cv adela navarrete.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Aaron Hernanz Moreno\cv aarón hernanz moreno 2015.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Aaron Piqueres Torres\curriculum aaron 2016.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Abel Ayuso\curriculum aspirante.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Abhir Hathiramani\cv_es_l1_.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Abigail Clemente Martin\cv abigail esp.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Abraham Alfaro\alfaro_abraham_cv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Adela de la Flor\cv. adela de la flor repetto..pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Adolfo Alcon Chanza\curriculum vitae.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Adriab Barroso\infojobs - adrián barroso arévalo cv.pdf
C:\User

Exception ignored in: <bound method Resource.__del__ of <wand.image.Image: (empty)>>
Traceback (most recent call last):
  File "C:\Users\fernando.coboaguiler\AppData\Local\Continuum\anaconda3\lib\site-packages\wand\resource.py", line 232, in __del__
    self.destroy()
  File "C:\Users\fernando.coboaguiler\AppData\Local\Continuum\anaconda3\lib\site-packages\wand\image.py", line 2767, in destroy
    for i in range(0, len(self.sequence)):
TypeError: object of type 'NoneType' has no len()
Exception ignored in: <bound method Resource.__del__ of <wand.image.Image: (empty)>>
Traceback (most recent call last):
  File "C:\Users\fernando.coboaguiler\AppData\Local\Continuum\anaconda3\lib\site-packages\wand\resource.py", line 232, in __del__
    self.destroy()
  File "C:\Users\fernando.coboaguiler\AppData\Local\Continuum\anaconda3\lib\site-packages\wand\image.py", line 2767, in destroy
    for i in range(0, len(self.sequence)):
TypeError: object of type 'NoneType' has no len()


C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alberto García\cv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alberto García Asenjo\cv alberto cunef.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alberto Garrote\cv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alberto Garzón\cv alberto garzón gonzález (3).pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alberto Gordejuela Domenech\alberto gordejuela doménech cv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alberto Guzman Aroca\alberto guzmán aroca- cv (español).pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\ALberto Hernandez Gonzalez\cv_alberto hernandez gonzalez.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alberto Linares\cv alberto linares obarrio.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alberto Lobato\cv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alberto Lopez Rodriguez\alberto lópez_c

C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alvaro Muñoz Úbeda\cv álvaro muñoz úbeda. 2015.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alvaro olivera guerrero\olivera guerrero álvaro cv castellano.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alvaro Perez Eguiluz\cv álvaro pérez.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alvaro Portes Navarro\cv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alvaro Prada Prada\cv álvaro prada.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alvaro Romero Olmos\álvaro romero olmos cv..pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alvaro Romo Herreo\curriculum_alvaroromoherrero.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alvaro San Hipolito\cvalvaroshl.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alvaro Sancerni Nozaleda\cv ávaro sancerni nozaleda.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Alvaro San

C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Ana Victoria Santos Borges\cv 01_16 ana victoria.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Ana Yebenes Retuerto\cv - ana yevenes retuerto.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Ana Zubiaur\cv ana zubiaur 11-17.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Anabel Reyes\curriculum vitae anabel.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Anabel Tomas y Benavent\cv at.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Ander Mugica Uranga\cv. ander mugica uranga.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Ander Vicario\ander vicario cv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Andra Lonela Matei\andra matei cv+expediente.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Andrea Aguilera Concepción\-cv andrea aguilera concepción foro.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Andrea Alonso 

...

C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Saverio Angelo Mancuso\cv saverio angelo mancuso (deloitte espana).pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sebastian Fernandez Medrano\cv 2015.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sebastian Freire Garcia\sfreiregarciacv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sergio Andreas Constantino Noguera\cv_sergio_andreas.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sergio Anel\curriculum vitae sergio anel vázquez.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sergio Antelo\currículum_vitae_sergio_antelo.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sergio Arenere Ruiz\cv_sergio arenere.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sergio Avila Testa\cv- avila testa, sergio.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sergio Babío\cv-sergiobabío.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs

Exception ignored in: <bound method Resource.__del__ of <wand.image.Image: (empty)>>
Traceback (most recent call last):
  File "C:\Users\fernando.coboaguiler\AppData\Local\Continuum\anaconda3\lib\site-packages\wand\resource.py", line 232, in __del__
    self.destroy()
  File "C:\Users\fernando.coboaguiler\AppData\Local\Continuum\anaconda3\lib\site-packages\wand\image.py", line 2767, in destroy
    for i in range(0, len(self.sequence)):
TypeError: object of type 'NoneType' has no len()


C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Simone Herrmann\cv_simone_herrmann.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sofia Alonso Duro\cv_sofía_alonso.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sofia Briganty\cv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sofia Fernandez Mariño\cv sofía fernández mariño.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sofia Jabon\cv sofía.jabon.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sofia Platas\sofía platas cv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sofía Díez\cv sofia diez.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sofía Gómez Villanueva\cv sgv.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sofía Martinez Molares\currículum vitae.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Sofía Morales López\cv_sofía morales lópez.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF

C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Victor Barriel Cantelar\curriculum.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Victor Botas Menendez\cv_victorbotas.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Victor Galvañ\cv-víctor-galvañ.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Victor Iriberri\cv victor iriberri leonor.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Victor Mahillo\cv victor mahillo.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Victor Manuel Madera Sanchez\cv vmms.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Victor Martinez Alvarez\cv-víctor_martínez_álvarez.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\victor men\cv_belenprovedo.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Victor Mendoza\cv víctor mendoza.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Victor Moran Abajo\curriculum victor moran.pdf
C:\Users\fernando.cob

C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Óscar Luis Ortega Gil\cv ortega gil, luis esp.pdf
C:\Users\fernando.coboaguiler\Desktop\TFM\Codigo\CVs_PDF\Óscar Martín\cv-óscar martín.pdf


### 3.3 Exportación de resultados a MongoDB

El código anterior ya se ha encargado de realizar el volcado de los chunks al repositorio de MongoDB